In [2]:
import os
import h5py
import gvar   as gv
import numpy  as np
import pandas as pd
from matplotlib import pyplot as plt

import corrfitter as cf

def format(v):
    return gv.gvar(
        v.mean(axis=0),
        v.std(axis=0) * np.sqrt(v.shape[0]-1)
    )
    
from b2heavy.ThreePointFunctions.types3pts import RatioFileString, RatioIO, Ratio, ratio_prerequisites, ratio_correction_factor
from b2heavy.TwoPointFunctions.utils import jkCorr
from b2heavy.FnalHISQMetadata import params

frm = '/Users/pietro/code/data_analysis/data/QCDNf2p1stag/B2heavy/lattice24'
datadir = '/Users/pietro/code/data_analysis/BtoD/Alex/'


ens = 'Coarse-1'
mom = '300'
jk  = 11

mdata = params(ens)
Ta,Tb = mdata['hSinks']

heavy = '_k' + mdata['kBStr']
light = '_k' + mdata['kDStr']
qq    = '_m' + mdata['mlStr']

data = h5py.File(f'/Users/pietro/code/data_analysis/BtoD/Alex/Ensembles/FnalHISQ/a0.12/{mdata["hdf5File2"]}')
data = data['data']

In [8]:
data3 = {}
for k in data.keys():
    st = k.split('_')
    if len(st)==12:
        src,cur,snk,tt,kh,s1,s2,s3,s4,mq,kl,pmom = st
        
        if snk in ['V2'] and \
            kh   == s_he   and \
            mq   == s_qq   and \
            kl   == s_li   and \
            pmom == 'p300' and cur[-1]==snk[-1]:

            smr = '1S' if s2=='1S' else 'd'
            data3[f'{src}.{cur}.{src}.{smr}.{tt}'] = jkCorr(data[k][:],bsize=jk)

In [12]:
for key in data3:
    src,cur,snk,smr,tt = key.split('.')

    mdl = cf.Corr3(
        datatag = key,
        T       = int(tt[-2:]),
        tmin    = 2,
        a       = ('an','ao'),
        b       = ('bn','bo'),
        dEa     = ('dEan','dEao'),
        dEb     = ('dEbn','dEbo'),
        sa      = (1,-1),
        sb      = (1,-1),
        Vnn='Vnn', Vno='Vno', Von='Von', Voo='Voo',
    )